## Генерация текстов
#### М.Горький "Трое", М.Горький "Старуха Изергиль"

##### Выполнили: Цендякова Светлана, Петрова Мария

In [45]:
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy as np

import re
import math


In [6]:
with open("troe.txt", "r", encoding="utf-8") as f:
    text = f.read()
    
print(text[:1500])

﻿Горький Максим Трое 
А.М.Горький Трое Среди лесов Керженца рассеяно много одиноких могил; в них тлеют кости старцев, людей древнего благочестия, об одном из таких старцев - Антипе - в деревнях, на Керженце, рассказывают: Суровый характером, богатый мужик Антипа Лунёв, дожив во грехе мирском до пятидесяти лет, задумался крепко, затосковал и, бросив семью, ушёл в леса. Там, на краю крутого оврага, он срубил себе келью и жил в ней восемь лет кряду, зиму и лето, не допуская к себе никого: ни знакомых, ни родных своих. Порою люди, заблудясь в лесу, случайно выходили к его келье и видели Антипу: он молился, стоя на коленях у порога её. Был он страшный: иссох в посте и молитве и весь, как зверь, оброс волосами. Завидев человека, он поднимался на ноги и молча кланялся ему до земли. Если его спрашивали, как выйти из леса, он без слов указывал рукою дорогу, ещё кланялся человеку до земли и, уходя в свою келью, запирался в ней. За восемь лет его видели часто, но никто никогда не слыхал его голос

##### Делаем предобработку

In [19]:
text = text.lower() #нижний регистр
words = [word for word in re.split('[ »–\—«\-\n\t\r,\.\?!:;\*\'\"“”\[\]\(\)/]', text) if word != ''] #сделали список слов
print('Количество слов', len(words))

Количество слов 80346


#### Сделаем поменьше, экономии времени ради

In [30]:
words = words[:25000]
tokens = set(words)
print('Количество слов', len(words))
print('Количество уникальных слов', len(tokens))

Количество слов 25000
Количество уникальных слов 7307


##### Далее присваиваем словам индексы, делаем таблицу соответствия слово-индекс, индекс-слово

In [21]:
nb_tokens = len(tokens)

token2index = dict((t, i) for i, t in enumerate(tokens))
index2token = dict((i, t) for i, t in enumerate(tokens))

##### Создаем входные строки и метки. Проходим по тексту с шагом=1 и выделяем отрезки SEQLEN=10. 

In [46]:
SEQLEN = 7
STEP = 1

input_tokens = []
label_tokens = []
for i in range(0, len(words) - SEQLEN, STEP):
    input_tokens.append(words[i:i + SEQLEN])
    label_tokens.append(words[i + SEQLEN]) #слова-метки
    
print('Входных токенов', len(input_tokens)) #количество различных возможных последовательностей
print('Слова-метки', len(label_tokens))

Входных токенов 24993
Слова-метки 24993


In [23]:
for i in range(10):
    print(' '.join(input_tokens[i]), '->', label_tokens[i])

﻿горький максим трое а м горький трое -> среди
максим трое а м горький трое среди -> лесов
трое а м горький трое среди лесов -> керженца
а м горький трое среди лесов керженца -> рассеяно
м горький трое среди лесов керженца рассеяно -> много
горький трое среди лесов керженца рассеяно много -> одиноких
трое среди лесов керженца рассеяно много одиноких -> могил
среди лесов керженца рассеяно много одиноких могил -> в
лесов керженца рассеяно много одиноких могил в -> них
керженца рассеяно много одиноких могил в них -> тлеют


##### Превращаем входные строки в тензоры и метки в векторы

In [24]:
X = np.zeros((len(input_tokens), SEQLEN, nb_tokens), dtype = np.bool)
y = np.zeros((len(input_tokens), nb_tokens), dtype = np.bool)

for i, input_tok(input_tokens):
    for j, t in enumerate(input_token):[i, j, token2index[t]] = 1
    y[i, token2index[label_tokens[i]]] = 1

#### Обучаем модель в течение 25 итераций по 1 эпохе, в конце каждой эпохи тестируем модель

In [25]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_tokens),
                    unroll=True))
model.add(Dense(nb_tokens))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [26]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    test_idx = np.random.randint(len(input_tokens))
    test_tokens = input_tokens[test_idx]
    print("Generating from seed: %s" % ' '.join(test_tokens))
    print(' '.join(test_tokens), end=" ")
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_tokens))
        for j, t in enumerate(test_tokens):
            Xtest[0, j, token2index[t]] = 1    #формируем вектор для Xtest
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2token[np.argmax(pred)]
        print(ypred, end=" ")
        
        test_tokens.append(ypred)    #добавляем предсказанное слово в последовательность
        test_tokens = test_tokens[1:]    #отбрасываем первое слово старой последовательности
    print()

Iteration #: 0
Epoch 1/1
24993/24993 [==============================] - 105s 4ms/step - loss: 7.7732
Generating from seed: друг друга не имея в этом надобности
друг друга не имея в этом надобности и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и 
Iteration #: 1
Epoch 1/1
24993/24993 [==============================] - 76s 3ms/step - loss: 7.4210
Generating from seed: ни о чём не думая слушал пение
ни о чём не думая слушал пение и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и 
Iteration #: 2
Epoch 1/1
24993/24993 [==============================] - 75s 3ms/step - loss: 7.1108
Generating from seed: на мальчика нехорошими глазами что сволочь здорово
на мальчика нехорошими глазами что сволочь з

24993/24993 [==============================] - 81s 3ms/step - loss: 4.9870
Generating from seed: приходила матица принося с собой булки чай
приходила матица принося с собой булки чай ему а потом не даже с еремея о сказал а то его тоже с не илья своей то он голову что в илье за и и её что в о был он с она а в за что было сказал и его у с с ним он с когда как из за потом его из однажды то не с еремей сказал илья илья он его на глядя и он он с видел что сказал с ему и с о ты с он но с с ну что с его он сказал ничего а он же что и бы и её 
Iteration #: 16
Epoch 1/1
24993/24993 [==============================] - 77s 3ms/step - loss: 4.8948
Generating from seed: жизнь в доме но илье захотелось узнать
жизнь в доме но илье захотелось узнать что она и то и он спросил яков но в на яков и я и что и что как тебе кто тоже что и так да и я в люди то и я тебе кто к кто как тебе и и так и него не илью время а не него что нет со и яков ну не мне ей вот ты мне а я уж это это тебе уж тебе ей ты что то чт

#### Загружаем другой текст автора, проводим предобработку по аналогии с тем, что делали в первый раз

In [36]:
with open("staruha_izergil.txt", "r", encoding="utf-8") as f:
    test_text = f.read()
    
print(test_text[:1500])

﻿




I


Я слышал эти рассказы под Аккерманом, в Бессарабии, на морском берегу.
Однажды вечером, кончив дневной сбор винограда, партия молдаван, с которой я работал, ушла на берег моря, а я и старуха Изергиль остались под густой тенью виноградных лоз и, лежа на земле, молчали, глядя, как тают в голубой мгле ночи силуэты тех людей, что пошли к морю.
Они шли, пели и смеялись; мужчины – бронзовые, с пышными, черными усами и густыми кудрями до плеч, в коротких куртках и широких шароварах; женщины и девушки – веселые, гибкие, с темно-синими глазами, тоже бронзовые. Их волосы, шелковые и черные, были распущены, ветер, теплый и легкий, играя ими, звякал монетами, вплетенными в них. Ветер тек широкой, ровной волной, но иногда он точно прыгал через что-то невидимое и, рождая сильный порыв, развевал волосы женщин в фантастические гривы, вздымавшиеся вокруг их голов. Это делало женщин странными и сказочными. Они уходили все дальше от нас, а ночь и фантазия одевали их все прекраснее.
Кто-то играл

In [37]:
test_text = test_text.lower()    #нижний регистр
test_words = [word for word in re.split('[ »\—«\-\n\t\r,\.\?!:;\*\'\"“”\[\]\(\)/]', test_text) if word != ''] #список слов


In [38]:
test_tokens = set(test_words)
print(len(test_tokens))

2576


In [39]:
test_inputs = []
test_labels = []
for i in range(0, len(test_words) - SEQLEN, STEP):
    test_inputs.append(test_words[i:i + SEQLEN])
    test_labels.append(test_words[i + SEQLEN])
    
print('Входные токены', len(test_inputs))
print('Слова-метки', len(test_labels))

Входные токены 7120
Слова-метки 7120


In [41]:
def perplexity(probs):
    logsum = 0
    for prob in probs:
        log_prob = math.log2(prob)
        logsum += log_prob
    l = logsum/len(probs)
    perplex = math.pow(2, -l)
    return perplex

In [42]:
def eval_model(model):
    probs = []
    for i in range(len(test_inputs)):
        test_input, test_label = test_inputs[i], test_labels[i]
        test_tensor = np.zeros((1, SEQLEN, nb_tokens))
    
        for j, t in enumerate(test_input):
            if t in token2index:  
                test_tensor[0, j, token2index[t]] = 1    #формируем тензор для каджой последовательности SEQLEN слов тестового текста
        #print('test_tensor.shape =', test_tensor.shape)
        red = model.predict(test_tensor, verbose=0)[0]
        #print(pred, '\n')
        if test_label in token2index:
            prob = pred[token2index[test_label]]
            probs.append(prob)
    prplx = perplexity(probs)
    return prplx

#### Перплексия

In [43]:
print(eval_model(model)) 

1168.5744765598174


### LSTM

In [47]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [48]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [49]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
24993/24993 [==============================] - 391s 16ms/step - loss: 7.8773

Epoch 00001: loss improved from inf to 7.87729, saving model to weights-improvement-01-7.8773.hdf5
Epoch 2/20
24993/24993 [==============================] - 385s 15ms/step - loss: 7.4185

Epoch 00002: loss improved from 7.87729 to 7.41850, saving model to weights-improvement-02-7.4185.hdf5
Epoch 3/20
24993/24993 [==============================] - 389s 16ms/step - loss: 7.3316

Epoch 00003: loss improved from 7.41850 to 7.33162, saving model to weights-improvement-03-7.3316.hdf5
Epoch 4/20
24993/24993 [==============================] - 380s 15ms/step - loss: 7.2624

Epoch 00004: loss improved from 7.33162 to 7.26243, saving model to weights-improvement-04-7.2624.hdf5
Epoch 5/20
24993/24993 [==============================] - 370s 15ms/step - loss: 7.1994

Epoch 00005: loss improved from 7.26243 to 7.19945, saving model to weights-improvement-05-7.1994.hdf5
Epoch 6/20
24993/24993 [====================

#### Подгружаем модель

In [51]:
filename = "weights-improvement-20-0.8139.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [62]:
int_to_char = dict((i, c) for i, c in enumerate(words)) #словарь соответствий, чтобы преобразовывать индексы в слова
int_to_char = tuple(int_to_char)

In [63]:
start = np.random.randint(0, len(X)-1)
pattern = X[start]
print( "Seed:")
print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print( "\nDone.")

Seed:


TypeError: only integer scalar arrays can be converted to a scalar index